<a href="https://colab.research.google.com/github/mridulmahajan04/CleaningCourseProject/blob/main/Topsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [177]:
def topsis(dataset, weight, impact, filename):
    try:
        if dataset is None or dataset.empty:
            raise ValueError("Dataset must not be empty.")
        if not weight or not impact or not filename:
            raise ValueError("Weights, impacts, and filename must be provided.")
        if not isinstance(weight, list) or not isinstance(impact, list):
            raise ValueError("Weights and impacts must be lists and seperated by comma.")
        if len(weight) != len(impact):
            raise ValueError("Number of weights and impacts must match the number of criteria.")
        if dataset.shape[1] - 1 != len(weight):
            raise ValueError("Number of weights must match the number of numeric columns in the dataset.")

        if not all(impact in ['+', '-'] for impact in impact):
            raise ValueError("Impacts must only contain '+' or '-'.")

        data = dataset.iloc[:, 1:].values
        normalized_data = data / np.sqrt((data**2).sum(axis=0))
        norm_data = normalized_data * weight

        ideal_case = np.where(np.array(impact) == '+', norm_data.max(axis=0), norm_data.min(axis=0))
        worst_case = np.where(np.array(impact) == '+', norm_data.min(axis=0), norm_data.max(axis=0))

        distance_best = np.sqrt(((norm_data - ideal_case)**2).sum(axis=1))
        distance_worst = np.sqrt(((norm_data - worst_case)**2).sum(axis=1))
        score = distance_worst / (distance_worst + distance_best)

        dataset['Performance Score'] = score
        dataset['Rank'] = dataset['Performance Score'].rank(ascending=False)
        dataset.sort_values(by='Rank', inplace=True)

        dataset.to_csv(filename, index=False)
        print(f"Results saved successfully to {filename}.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [178]:
import pandas as pd
import numpy as np

try:
    file_path = '102396003-data.xlsx'
    data = pd.read_excel(file_path)

    data.to_csv('102396003(data).csv', index=False)
    datasets = pd.read_csv('102396003(data).csv')

    if datasets.shape[1] < 3:
        raise ValueError("Input file must contain at least three columns.")
    if not np.issubdtype(datasets.iloc[:, 1:].dtypes.values[0], np.number):
        raise ValueError("From the 2nd to last columns, all values must be numeric.")

    weight = [1, 1, 1, 2, 1]
    impact = ['+', '+', '-', '+', '+']

    topsis(datasets, weight, impact, "102396003_result.csv")

except FileNotFoundError:
    print(f"File not found: {file_path}. Please ensure the file exists and the path is correct.")
except Exception as e:
    print(f"An error occurred: {e}")


Results saved successfully to 102396003_result.csv.
